<a href="https://colab.research.google.com/github/JuliPolu/Deep_Learning_TensorFlow_Course/blob/main/Assignment_4_CNN_PJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание. Урок 4.

## Практическое задание

Домашнее задание
обучить рукописную сверточную нейронную сеть (с падением размера ядра свертки и последовательностью блоков свертка-пулинг (conv-pool)-(conv-pool)-...) на датасете cifar-10 <br>
оценить рост точности при увеличении ширины сети (больше фильтров) <br>
оценить рост точности при увеличении глубины сети (больше слоев) <br>
*сравнить с точностью полносвязной сети для этой выборки <br>

In [ ]:
from __future__ import print_function
import tensorflow.keras as keras # расскоментируйте эту строку, чтобы начать обучение
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import os

In [ ]:
import pandas as pd

In [ ]:
classes = ['самолет', 'автомобиль', 'птица', 'кот', 'олень', 'собака', 'лягушка', 'лошадь', 'корабль', 'грузовик']


## Установка параметров нейросети

In [ ]:
batch_size = 32
num_classes = 10
epochs = 10
data_augmentation = False
num_predictions = 20
# save_dir = os.path.join(os.getcwd(), 'saved_models')
# model_name = 'keras_cifar10_trained_model.h5'


## Разделение тренировочной и тестовой выборки

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'тренировочные примеры')
print(x_test.shape[0], 'тестовые примеры')

170498071/170498071 [==============================] - 2s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры


## Преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1

In [ ]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255



## Конфигурирование нейросети

In [ ]:
def bild_model(koef_core = 1, num_layers = 1):
    model = Sequential()

    # слои нейросети отвественные за свертку и max-pooling
    model.add(Conv2D(32 * koef_core, (3, 3), padding='same', input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))

    for _ in range(num_layers):
        model.add(Conv2D(32 * koef_core, (3, 3)))
        model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64 * koef_core, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64 * koef_core, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Dropout(0.25))

    # полносвязные слои нейронной сети
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # компиляция модели
    model.compile(loss='categorical_crossentropy',
                  optimizer='SGD',
                  metrics=['accuracy'])

    return model

In [ ]:
result = []
for koef_core in [1, 2]:
  for num_layers in [1,2]:
    model = bild_model(koef_core=koef_core, num_layers=num_layers)
    history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
    scores = model.evaluate(x_test, y_test, verbose=1)
    result.append([koef_core, num_layers, history.history['loss'][-1], history.history['accuracy'][-1], scores[0], scores[1]])

Epoch 1/10
1563/1563 [==============================] - 19s 6ms/step - loss: 2.1318 - accuracy: 0.2043 - val_loss: 1.8659 - val_accuracy: 0.3337
Epoch 2/10
1563/1563 [==============================] - 10s 6ms/step - loss: 1.7672 - accuracy: 0.3566 - val_loss: 1.6805 - val_accuracy: 0.3861
Epoch 3/10
1563/1563 [==============================] - 10s 7ms/step - loss: 1.6203 - accuracy: 0.4097 - val_loss: 1.4772 - val_accuracy: 0.4652
Epoch 4/10
1563/1563 [==============================] - 10s 7ms/step - loss: 1.5207 - accuracy: 0.4495 - val_loss: 1.4336 - val_accuracy: 0.4785
Epoch 5/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.4256 - accuracy: 0.4863 - val_loss: 1.3090 - val_accuracy: 0.5301
Epoch 6/10
1563/1563 [==============================] - 10s 7ms/step - loss: 1.3503 - accuracy: 0.5130 - val_loss: 1.2529 - val_accuracy: 0.5561
Epoch 7/10
1563/1563 [==============================] - 9s 6ms/step - loss: 1.2878 - accuracy: 0.5375 - val_loss: 1.2488 - val_acc

In [ ]:
# проверка работы обученной модели
pd.DataFrame(result, columns=['core', 'layers', 'loss train','accuracy train','loss test','accuracy test'])

,core,layers,loss train,accuracy train,loss test,accuracy test
0,1,1,1.138409,0.59396,1.025703,0.6398
1,1,2,1.116525,0.60504,1.010669,0.6469
2,2,1,0.967390,0.65776,0.897607,0.6863
3,2,2,1.011961,0.64084,0.941022,0.6692
